In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
file_path = "E:/Data/South Korea/South korea.csv"
df = pd.read_csv(file_path,parse_dates=["Date"],index_col=[0])

In [ ]:
df = scaler.fit_transform(df)
# “Initialize an empty list to store data from different stages.”
stages = []
# Set up according to the actual stage division. Here is an example.
intervals = [(0, 199), (200, 719), (720, 821), (822, 879), (880, 920)]
for start, end in intervals:
    stages.append(df[start:end, :])
for i, stage in enumerate(stages):
    print(f"Stage {i+1} shape: {stage.shape}")

In [ ]:
from tensorflow.keras.layers import Input, Dense, Attention
from tensorflow.keras.models import Model

# “Customize the self-attention layer.”
class SelfAttention(tf.keras.layers.Layer):
    def __init__(self, input_dim):
        super(SelfAttention, self).__init__()
        self.query_dense = Dense(input_dim)
        self.key_dense = Dense(input_dim)
        self.value_dense = Dense(input_dim)
        self.softmax = tf.keras.layers.Softmax(axis=-1)

    def call(self, inputs):
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        attention_scores = tf.matmul(query, key, transpose_b=True) / tf.sqrt(tf.cast(tf.shape(key)[-1], tf.float32))
        attention_weights = self.softmax(attention_scores)
        output = tf.matmul(attention_weights, value)
        return output, attention_weights

In [ ]:
stages = [stages[0],stages[1],stages[2],stages[3],stages[4]]
# “Create a self-attention layer.”
input_dim = 19
attention_layer = SelfAttention(input_dim)

# Calculate the correlation between each stage.
attention_weights_list = []
for stage in stages:
    stage_input = tf.convert_to_tensor(stage, dtype=tf.float32)
    _, attention_weights = attention_layer(stage_input)
    attention_weights_list.append(attention_weights)

stage_weights = [tf.reduce_mean(weights).numpy() for weights in attention_weights_list]

In [ ]:
stage0=stages[0]*stage_weights[0]
stage1 = stages[1]*stage_weights[1]
stage2=stages[2]*stage_weights[2]
stage3 = stages[3]*stage_weights[3]
stage4=stages[4]*stage_weights[4]

In [ ]:
train_=tf.concat([stage0,stage1,stage2,stage3,stage4], axis=0)
#You need to save the Weighted data,you can choose the path according to your actual situation.
train_.to_csv('E:/Data/weighted data')